In [1]:
import xarray as xr
import xesmf as xe

wind = xr.open_dataset('data/nwp_vorticity_197901_202508.nc')
ssta = xr.open_dataset('data/nwp_ssta_197901_202508.nc')

wind = wind.drop_vars(['latitude', 'longitude'])
ssta = ssta.rename(
    {
        'longitude': 'lon',
        'latitude': 'lat',
    }
)

wind_regrid = xr.Dataset(
    {
        'lat': (['lat'], ssta.lat.data, {'units': 'degrees_north'}),
        'lon': (['lon'], ssta.lon.data, {'units': 'degrees_east'}),
    }
)

regridder = xe.Regridder(wind, wind_regrid, 'conservative')

uwind_regrid = regridder(wind.uwind, keep_attrs=True)
vwind_regrid = regridder(wind.vwind, keep_attrs=True)
vort_regrid = regridder(wind.vorticity, keep_attrs=True)

time_ref = ssta.time

uwnd_aligned = uwind_regrid.interp(time=time_ref)
vwnd_aligned = vwind_regrid.interp(time=time_ref)
vort_aligned = vort_regrid.interp(time=time_ref)

ds = xr.Dataset(
    data_vars={
        'ssta': ssta['sst'],
        'uwnd': uwnd_aligned,
        'vwnd': vwnd_aligned,
        'vorticity': vort_aligned,
    },
    coords={'time': time_ref, 'lon': ssta.lon, 'lat': ssta.lat}
)

ds.to_netcdf('data/nwp_regrid_197901_202508.nc')